In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertModel
import torch
import tensorflow as tf
from imblearn.over_sampling import SMOTE



c:\Users\Gayathri Anil\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load your dataset with Malayalam and Malayalam-English mixed comments
train_data = pd.read_csv(r"C:\Users\Gayathri Anil\OneDrive\Desktop\abusive comment\mal_full_offensive_train (1).csv",delimiter="\t", header= None)
test_data = pd.read_csv(r"C:\Users\Gayathri Anil\OneDrive\Desktop\abusive comment\mal_full_offensive_test (1).csv",delimiter="\t", header= None)
val_data=pd.read_csv(r"C:\Users\Gayathri Anil\OneDrive\Desktop\abusive comment\mal_full_offensive_dev (3).csv",delimiter="\t", header= None)

In [3]:
train_data

,0,1,2
0,പലദേശം. പല ഭാഷ ഒരേ ഒരു രാജാവ് അല്ലാതെ സ്വന്ത...,Not_offensive,NaN
1,ഈ ഓണം ഏട്ടനും പിള്ളേർക്ക് ഉള്ളതാണ് എന്ന് ഉള്ളവ...,Not_offensive,NaN
2,ആരണ്ട ആരണ്ട തലുണ്ടാകാണാ ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്...,Not_offensive,NaN
3,Sushin syam Shaiju khalid Midhun manual,Not_offensive,NaN
4,J A K E S. B EJ O Y !!!,Not_offensive,NaN
...,...,...,...
16005,കട്ട ലാലേട്ടൻ ഫാൻസിന് ദൈവത്തെ ഓർത്ത് അമിത പ്രത...,Not_offensive,NaN
16006,ente mammookka ningal puliyalla oru simhama......,Not_offensive,NaN
16007,Lucifer mass dialogues Ellam onnu comment chey...,Not_offensive,NaN
16008,Like from Madurai (Tamil nadu) ....,not-malayalam,NaN


In [4]:
val_data

,0,1,2
0,Gopichettante BGM um mammookayum ishtapedunnav...,Not_offensive,NaN
1,ഇത് ഒരു പെണ്ണ് തന്നെ ആണോ direct ചെയ്യുന്നത് p...,Not_offensive,NaN
2,പൃഥ്വിരാജ് സുരാജേട്ടൻ ലാലും അലക്സ്.. Lal jr. ന...,Not_offensive,NaN
3,പോകരുത് മക്കളെ പോക്ക .......... നൻ കണ്ട് എന്റ്...,Not_offensive,NaN
4,avan Varum ente Makan Madhura Raja....,Not_offensive,NaN
...,...,...,...
1994,Onnam ഇട്ടി മണി & brathors ഡേ വിന്നർ ലാലേട്ടൻ ...,Not_offensive,NaN
1995,Manju chechi undel hero vendaa. Chechi vere le...,Not_offensive,NaN
1996,Ith kollaaaam....kanda ooodayp trailer ne kalu...,Not_offensive,NaN
1997,Ufffff. Pwoli lalattaa... Katta waiting,Not_offensive,NaN


In [5]:
train_data[1].value_counts()

1
Not_offensive                           14153
not-malayalam                            1287
Offensive_Targeted_Insult_Individual      239
Offensive_Untargetede                     191
Offensive_Targeted_Insult_Group           140
Name: count, dtype: int64

In [6]:
test_data[1].value_counts()

1
Not_offensive                           1765
not-malayalam                            157
Offensive_Untargetede                     29
Offensive_Targeted_Insult_Individual      27
Offensive_Targeted_Insult_Group           23
Name: count, dtype: int64

In [7]:
val_data[1].value_counts()

1
Not_offensive                           1779
not-malayalam                            163
Offensive_Targeted_Insult_Individual      24
Offensive_Untargetede                     20
Offensive_Targeted_Insult_Group           13
Name: count, dtype: int64

In [8]:
test_data

,0,1,2
0,അപ്പൊ ഇതൊരൊന്നൊരാ മൊതലാണല്ലേ Suraj ആണ് നടൻ ന്...,Not_offensive,NaN
1,എന്ത് ഊള എഡിറ്റിംഗ് ആടോ ഇത് ഒരുമാതിരി vivo vid...,Not_offensive,NaN
2,Fefka ee padam release cheyyan samadhicho?,Not_offensive,NaN
3,അആഹാ.. സംഗീതം ജെക്‌സ് ബിജോയ് ആണ് അപ്പൊ പൊട്ടലു...,Not_offensive,NaN
4,Ravile thane views likes ethra ayyi enn nokan ...,Not_offensive,NaN
...,...,...,...
1996,Swargatthil ninnu purathaakkappetta daivatthin...,Not_offensive,NaN
1997,Ivide Palakkad Jayettan Fans club nnu ashamsak...,Not_offensive,NaN
1998,ഈ പടത്തിന് വെയിറ്റ് ചെയ്യുന്ന മമ്മൂക്ക ഫാൻസും,Not_offensive,NaN
1999,കട്ട ലാലേട്ടൻ ഫാൻസ് ഒരു ലൈക് തന്നിട്ട് പോവാമോ ...,Not_offensive,NaN


In [9]:
train_data.dropna(axis=1, inplace=True)


In [10]:
test_data.dropna(axis=1, inplace=True)


In [11]:
val_data.dropna(axis=1, inplace=True)


In [12]:
train_data

,0,1
0,പലദേശം. പല ഭാഷ ഒരേ ഒരു രാജാവ് അല്ലാതെ സ്വന്ത...,Not_offensive
1,ഈ ഓണം ഏട്ടനും പിള്ളേർക്ക് ഉള്ളതാണ് എന്ന് ഉള്ളവ...,Not_offensive
2,ആരണ്ട ആരണ്ട തലുണ്ടാകാണാ ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്...,Not_offensive
3,Sushin syam Shaiju khalid Midhun manual,Not_offensive
4,J A K E S. B EJ O Y !!!,Not_offensive
...,...,...
16005,കട്ട ലാലേട്ടൻ ഫാൻസിന് ദൈവത്തെ ഓർത്ത് അമിത പ്രത...,Not_offensive
16006,ente mammookka ningal puliyalla oru simhama......,Not_offensive
16007,Lucifer mass dialogues Ellam onnu comment chey...,Not_offensive
16008,Like from Madurai (Tamil nadu) ....,not-malayalam


In [13]:
test_data

,0,1
0,അപ്പൊ ഇതൊരൊന്നൊരാ മൊതലാണല്ലേ Suraj ആണ് നടൻ ന്...,Not_offensive
1,എന്ത് ഊള എഡിറ്റിംഗ് ആടോ ഇത് ഒരുമാതിരി vivo vid...,Not_offensive
2,Fefka ee padam release cheyyan samadhicho?,Not_offensive
3,അആഹാ.. സംഗീതം ജെക്‌സ് ബിജോയ് ആണ് അപ്പൊ പൊട്ടലു...,Not_offensive
4,Ravile thane views likes ethra ayyi enn nokan ...,Not_offensive
...,...,...
1996,Swargatthil ninnu purathaakkappetta daivatthin...,Not_offensive
1997,Ivide Palakkad Jayettan Fans club nnu ashamsak...,Not_offensive
1998,ഈ പടത്തിന് വെയിറ്റ് ചെയ്യുന്ന മമ്മൂക്ക ഫാൻസും,Not_offensive
1999,കട്ട ലാലേട്ടൻ ഫാൻസ് ഒരു ലൈക് തന്നിട്ട് പോവാമോ ...,Not_offensive


In [14]:
val_data

,0,1
0,Gopichettante BGM um mammookayum ishtapedunnav...,Not_offensive
1,ഇത് ഒരു പെണ്ണ് തന്നെ ആണോ direct ചെയ്യുന്നത് p...,Not_offensive
2,പൃഥ്വിരാജ് സുരാജേട്ടൻ ലാലും അലക്സ്.. Lal jr. ന...,Not_offensive
3,പോകരുത് മക്കളെ പോക്ക .......... നൻ കണ്ട് എന്റ്...,Not_offensive
4,avan Varum ente Makan Madhura Raja....,Not_offensive
...,...,...
1994,Onnam ഇട്ടി മണി & brathors ഡേ വിന്നർ ലാലേട്ടൻ ...,Not_offensive
1995,Manju chechi undel hero vendaa. Chechi vere le...,Not_offensive
1996,Ith kollaaaam....kanda ooodayp trailer ne kalu...,Not_offensive
1997,Ufffff. Pwoli lalattaa... Katta waiting,Not_offensive


In [15]:
train_data.isnull().sum()


0    0
1    0
dtype: int64

In [16]:
test_data.isnull().sum()


0    0
1    0
dtype: int64

In [17]:
val_data.isnull().sum()


0    0
1    0
dtype: int64

In [18]:
train_data.columns

Index([0, 1], dtype='int64')

In [19]:
test_data.columns

Index([0, 1], dtype='int64')

In [20]:
val_data.columns

Index([0, 1], dtype='int64')

In [21]:
train_data[1] = train_data[1].replace({"not-malayalam":np.NaN})


In [22]:
test_data[1] = train_data[1].replace({"not-malayalam":np.NaN})


In [23]:
val_data[1] = train_data[1].replace({"not-malayalam":np.NaN})


In [24]:
train_data.isnull().sum()

0       0
1    1287
dtype: int64

In [25]:
test_data.isnull().sum()

0      0
1    152
dtype: int64

In [26]:
val_data.isnull().sum()

0      0
1    152
dtype: int64

In [27]:
train_data.dropna(axis=0,how='any', inplace=True)

In [28]:
test_data.dropna(axis=0,how='any', inplace=True)

In [29]:
val_data.dropna(axis=0,how='any', inplace=True)

In [30]:
train_data

,0,1
0,പലദേശം. പല ഭാഷ ഒരേ ഒരു രാജാവ് അല്ലാതെ സ്വന്ത...,Not_offensive
1,ഈ ഓണം ഏട്ടനും പിള്ളേർക്ക് ഉള്ളതാണ് എന്ന് ഉള്ളവ...,Not_offensive
2,ആരണ്ട ആരണ്ട തലുണ്ടാകാണാ ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്...,Not_offensive
3,Sushin syam Shaiju khalid Midhun manual,Not_offensive
4,J A K E S. B EJ O Y !!!,Not_offensive
...,...,...
16004,ഈ ഓണത്തിന് എങ്ങനെ മുണ്ട് എളുപ്പത്തിൽ ഉടുക്കാം ...,Not_offensive
16005,കട്ട ലാലേട്ടൻ ഫാൻസിന് ദൈവത്തെ ഓർത്ത് അമിത പ്രത...,Not_offensive
16006,ente mammookka ningal puliyalla oru simhama......,Not_offensive
16007,Lucifer mass dialogues Ellam onnu comment chey...,Not_offensive


In [31]:
test_data

,0,1
0,അപ്പൊ ഇതൊരൊന്നൊരാ മൊതലാണല്ലേ Suraj ആണ് നടൻ ന്...,Not_offensive
1,എന്ത് ഊള എഡിറ്റിംഗ് ആടോ ഇത് ഒരുമാതിരി vivo vid...,Not_offensive
2,Fefka ee padam release cheyyan samadhicho?,Not_offensive
3,അആഹാ.. സംഗീതം ജെക്‌സ് ബിജോയ് ആണ് അപ്പൊ പൊട്ടലു...,Not_offensive
4,Ravile thane views likes ethra ayyi enn nokan ...,Not_offensive
...,...,...
1996,Swargatthil ninnu purathaakkappetta daivatthin...,Not_offensive
1997,Ivide Palakkad Jayettan Fans club nnu ashamsak...,Not_offensive
1998,ഈ പടത്തിന് വെയിറ്റ് ചെയ്യുന്ന മമ്മൂക്ക ഫാൻസും,Not_offensive
1999,കട്ട ലാലേട്ടൻ ഫാൻസ് ഒരു ലൈക് തന്നിട്ട് പോവാമോ ...,Not_offensive


In [32]:
val_data

,0,1
0,Gopichettante BGM um mammookayum ishtapedunnav...,Not_offensive
1,ഇത് ഒരു പെണ്ണ് തന്നെ ആണോ direct ചെയ്യുന്നത് p...,Not_offensive
2,പൃഥ്വിരാജ് സുരാജേട്ടൻ ലാലും അലക്സ്.. Lal jr. ന...,Not_offensive
3,പോകരുത് മക്കളെ പോക്ക .......... നൻ കണ്ട് എന്റ്...,Not_offensive
4,avan Varum ente Makan Madhura Raja....,Not_offensive
...,...,...
1994,Onnam ഇട്ടി മണി & brathors ഡേ വിന്നർ ലാലേട്ടൻ ...,Not_offensive
1995,Manju chechi undel hero vendaa. Chechi vere le...,Not_offensive
1996,Ith kollaaaam....kanda ooodayp trailer ne kalu...,Not_offensive
1997,Ufffff. Pwoli lalattaa... Katta waiting,Not_offensive


In [33]:
train_data[1] = [0 if i =="Not_offensive" else 1 for i in train_data[1]]

In [34]:
train_data.head()

,0,1
0,പലദേശം. പല ഭാഷ ഒരേ ഒരു രാജാവ് അല്ലാതെ സ്വന്ത...,0
1,ഈ ഓണം ഏട്ടനും പിള്ളേർക്ക് ഉള്ളതാണ് എന്ന് ഉള്ളവ...,0
2,ആരണ്ട ആരണ്ട തലുണ്ടാകാണാ ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്...,0
3,Sushin syam Shaiju khalid Midhun manual,0
4,J A K E S. B EJ O Y !!!,0


In [35]:
test_data[1] = [0 if i =="Not_offensive" else 1 for i in test_data[1]]

In [36]:
test_data

,0,1
0,അപ്പൊ ഇതൊരൊന്നൊരാ മൊതലാണല്ലേ Suraj ആണ് നടൻ ന്...,0
1,എന്ത് ഊള എഡിറ്റിംഗ് ആടോ ഇത് ഒരുമാതിരി vivo vid...,0
2,Fefka ee padam release cheyyan samadhicho?,0
3,അആഹാ.. സംഗീതം ജെക്‌സ് ബിജോയ് ആണ് അപ്പൊ പൊട്ടലു...,0
4,Ravile thane views likes ethra ayyi enn nokan ...,0
...,...,...
1996,Swargatthil ninnu purathaakkappetta daivatthin...,0
1997,Ivide Palakkad Jayettan Fans club nnu ashamsak...,0
1998,ഈ പടത്തിന് വെയിറ്റ് ചെയ്യുന്ന മമ്മൂക്ക ഫാൻസും,0
1999,കട്ട ലാലേട്ടൻ ഫാൻസ് ഒരു ലൈക് തന്നിട്ട് പോവാമോ ...,0


In [37]:
val_data[1] = [0 if i =="Not_offensive" else 1 for i in val_data[1]]

In [38]:
val_data

,0,1
0,Gopichettante BGM um mammookayum ishtapedunnav...,0
1,ഇത് ഒരു പെണ്ണ് തന്നെ ആണോ direct ചെയ്യുന്നത് p...,0
2,പൃഥ്വിരാജ് സുരാജേട്ടൻ ലാലും അലക്സ്.. Lal jr. ന...,0
3,പോകരുത് മക്കളെ പോക്ക .......... നൻ കണ്ട് എന്റ്...,0
4,avan Varum ente Makan Madhura Raja....,0
...,...,...
1994,Onnam ഇട്ടി മണി & brathors ഡേ വിന്നർ ലാലേട്ടൻ ...,0
1995,Manju chechi undel hero vendaa. Chechi vere le...,0
1996,Ith kollaaaam....kanda ooodayp trailer ne kalu...,0
1997,Ufffff. Pwoli lalattaa... Katta waiting,0


In [39]:
X_train=train_data[0]
y_train=train_data[1]

In [40]:
X_test=test_data[0]
y_test=test_data[1]

In [41]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

In [42]:
# Tokenize the text data and extract mBERT embeddings
X_train_embeddings = []
X_test_embeddings = []

In [43]:
for text in X_train:
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**tokens)
    embeddings = model_output['last_hidden_state'].mean(dim=1).squeeze().numpy()
    X_train_embeddings.append(embeddings)

In [44]:
for text in X_test:
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**tokens)
    embeddings = model_output['last_hidden_state'].mean(dim=1).squeeze().numpy()
    X_test_embeddings.append(embeddings)

In [45]:
# Convert the embeddings to numpy arrays
X_train_embeddings = np.array(X_train_embeddings)
X_test_embeddings = np.array(X_test_embeddings)

In [48]:
print(X_train_embeddings.shape, y_train.shape)


(14723, 768) (14723,)


In [49]:
y_train = y_train.values.reshape(-1, 1)  # Reshape if y_train is a pandas Series


In [62]:
# Apply SMOTE to balance the training set
smote = SMOTE(sampling_strategy='auto',random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_embeddings, y_train)


# Convert the resampled data back to DataFrame format
X_train_resampled = pd.Series(X_train_resampled.flatten())
y_train_resampled = pd.Series(y_train_resampled)

In [63]:
# Apply SMOTE to balance the testing set
smote = SMOTE(sampling_strategy='auto',random_state=42)
X_test_resampled, y_test_resampled = smote.fit_resample(X_test_embeddings, y_test)

# Convert the resampled data back to DataFrame format
X_test_resampled = pd.Series(X_test_resampled.flatten())
y_test_resampled = pd.Series(y_test_resampled)

In [64]:
print(X_train_resampled.shape, y_train_resampled.shape)


(21739008,) (28306,)


In [65]:
from sklearn.tree import DecisionTreeClassifier
d_classifier = DecisionTreeClassifier(random_state=42)
d_classifier.fit(X_train_embeddings, y_train)

DecisionTreeClassifier(random_state=42)

In [66]:
y_pred = d_classifier.predict(X_test_embeddings)


In [67]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [68]:
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Classification Report:\n", report)

Accuracy: 91.13%
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.94      0.95      1783
           1       0.05      0.08      0.06        66

    accuracy                           0.91      1849
   macro avg       0.51      0.51      0.51      1849
weighted avg       0.93      0.91      0.92      1849



In [69]:
import pickle
saved_model=pickle.dumps(d_classifier)

In [70]:
b= pickle.loads(saved_model)

In [71]:
note = ['Jacket..Cooling glass...Car...Slow motion...Pazhaya bomb katha tanne....Hahahaha ..Eyalku madukille ethonnum...']
tokens = tokenizer(note, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    model_output = model(**tokens)
embeddings = model_output['last_hidden_state'].mean(dim=1).squeeze().numpy()


input_embedding = embeddings.reshape(1, -1)  

predicted_label = d_classifier.predict(input_embedding)

print("Predicted Label:", predicted_label)

Predicted Label: [0]
